### Changing to the main directory

In [1]:
%cd ..

/home/isham/Desktop/machine-learning-projects/text-classification/few-shot-prompting-text-classification


/home/isham/anaconda3/envs/text-classification-using-llm/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Import Libraries

In [2]:
import pandas as pd
from utils import split_csv

### Loading the dataset

In [3]:
df = pd.read_csv("./data/news_category_classification/bbc-text.csv")

In [4]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [6]:
# Checking the number of categories and if the imbalance of the dataset is severe
df['category'].value_counts()

category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

### Preparation for Few Shot Examples

Selecting 2 examples from each category for the few shot examples.

In [7]:
df.sample(n=10, random_state=42)

,category,text
414,politics,brown and blair face new rift claims for the u...
420,business,small firms hit by rising costs rising fuel ...
1644,entertainment,spirit awards hail sideways the comedy sideway...
416,tech,microsoft releases patches microsoft has warne...
1232,sport,arsenal through on penalties arsenal win 4-2 o...
1544,business,jobs go at oracle after takeover oracle has an...
1748,business,id theft surge hits us consumers almost a quar...
1264,sport,poll explains free-kick decision referee graha...
629,sport,parmar ruled out of davis cup tie a knee injur...
1043,tech,video phones act as dating tools technologies ...


In [8]:
df.sample(n=10, random_state=41)

,category,text
623,sport,bryan twins keep us hopes alive the united sta...
2157,sport,woodward eyes brennan for lions toulouse s for...
591,politics,david blunkett in quotes david blunkett - who ...
112,sport,prodigy monfils blows away gaudio french prodi...
1214,tech,toxic web links help virus spread virus writer...
1912,entertainment,elvis fans hold birthday bash elvis fans aroun...
1056,sport,wenger steps up row arsene wenger has stepped ...
1461,entertainment,singer ian brown in gig arrest former stone ...
1019,sport,boro suffer morrison injury blow middlesbrough...
396,business,tsunami to hit sri lanka banks sri lanka s b...


In [9]:
index_locs_to_be_filtered = [414, 591, 420, 396, 1912, 1461, 1056, 1019, 416, 1214]

In [10]:
few_shot_examples_df = df.iloc[index_locs_to_be_filtered]
few_shot_examples_df.reset_index(drop=True, inplace=True)

In [11]:
few_shot_examples_df

,category,text
0,politics,brown and blair face new rift claims for the u...
1,politics,david blunkett in quotes david blunkett - who ...
2,business,small firms hit by rising costs rising fuel ...
3,business,tsunami to hit sri lanka banks sri lanka s b...
4,entertainment,elvis fans hold birthday bash elvis fans aroun...
5,entertainment,singer ian brown in gig arrest former stone ...
6,sport,wenger steps up row arsene wenger has stepped ...
7,sport,boro suffer morrison injury blow middlesbrough...
8,tech,microsoft releases patches microsoft has warne...
9,tech,toxic web links help virus spread virus writer...


In [12]:
user_prompt_template = f"""Examples on how the news article as input text is classified:
```
Input: {few_shot_examples_df['text'][1]}
Output: {few_shot_examples_df['category'][1]}

Input: {few_shot_examples_df['text'][2]}
Output: {few_shot_examples_df['category'][2]}

Input: {few_shot_examples_df['text'][4]}
Output: {few_shot_examples_df['category'][4]}

Input: {few_shot_examples_df['text'][6]}
Output: {few_shot_examples_df['category'][6]}

Input: {few_shot_examples_df['text'][8]}
Output: {few_shot_examples_df['category'][8]}
```

Now, classify the following text. Respond with one word from the five categories: Sport, Tech, Business, Entertainment, or Politics.:

Input: {{text_to_be_classified}}
Output:
"""

In [13]:
with open("./data/news_category_classification/user_prompt_template2.txt", "w") as f:
    f.write(user_prompt_template)

### Preparation for Text Examples

For fair evaluation, we will exclude out the few-shot examples from the dataset. And then only proceed with selecting test examples.

In [14]:
new_df = df.drop(index=index_locs_to_be_filtered)
new_df.reset_index(drop=True, inplace=True)
new_df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,business,cars pull down us retail figures us retail sal...
2211,politics,kilroy unveils immigration policy ex-chatshow ...
2212,entertainment,rem announce new glasgow concert us band rem h...
2213,politics,how political squabbles snowball it s become c...


In [15]:
new_df.sample(n=500, random_state=42)['category'].value_counts()

category
business         118
sport            108
politics         100
entertainment     88
tech              86
Name: count, dtype: int64

In [16]:
text_examples = new_df.sample(n=500, random_state=42)
text_examples.reset_index(drop=True, inplace=True)

In [17]:
text_examples.to_csv("./data/news_category_classification/test-examples.csv", index=False)

Splitting the single CSV file into multiple CSV files with 50 rows

In [18]:
split_csv('./data/news_category_classification/test-examples.csv', 50, './data/news_category_classification/test-examples')

2024-05-24 15:37:22,334 - INFO - Successfully read ./data/news_category_classification/test-examples.csv
2024-05-24 15:37:22,334 - INFO - Total rows in file: 500. Rows per split: 50. Total splits needed: 10
2024-05-24 15:37:22,336 - INFO - Written ./data/news_category_classification/test-examples_1.csv with rows 1 to 50


2024-05-24 15:37:22,338 - INFO - Written ./data/news_category_classification/test-examples_2.csv with rows 51 to 100
2024-05-24 15:37:22,340 - INFO - Written ./data/news_category_classification/test-examples_3.csv with rows 101 to 150
2024-05-24 15:37:22,342 - INFO - Written ./data/news_category_classification/test-examples_4.csv with rows 151 to 200
2024-05-24 15:37:22,343 - INFO - Written ./data/news_category_classification/test-examples_5.csv with rows 201 to 250
2024-05-24 15:37:22,345 - INFO - Written ./data/news_category_classification/test-examples_6.csv with rows 251 to 300
2024-05-24 15:37:22,347 - INFO - Written ./data/news_category_classification/test-examples_7.csv with rows 301 to 350
2024-05-24 15:37:22,348 - INFO - Written ./data/news_category_classification/test-examples_8.csv with rows 351 to 400
2024-05-24 15:37:22,350 - INFO - Written ./data/news_category_classification/test-examples_9.csv with rows 401 to 450
2024-05-24 15:37:22,352 - INFO - Written ./data/news_cate